In [ ]:
import pandas as pd
import requests
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

In [ ]:
##注意！每次使用当前block会重置数据集
train_data = pd.DataFrame(columns=['year','title','author','abstract','link','pdf'])
train_data.to_csv('ICML.csv')
#读取现在有的数据集
dataset=pd.read_csv('ICML.csv')

In [ ]:
#设置虚拟浏览器。
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
driver = webdriver.Chrome(chrome_options=chrome_options)
#driver = webdriver.Chrome()

In [ ]:
def spider_ICML(year,web):
    html=requests.get(web)
    pattern = re.compile(r'http://proceedings.mlr.press/[0-9a-zA-Z]+/[0-9a-zA-Z]+.html')
    result1 = pattern.findall(html.text)
    lists = list(set(result1))
    wrong = range(len(lists))
    t = 0
    while True:
        print('第'+str(year)+'年的第'+str(t)+'次循环')
        localtime = time.asctime( time.localtime(time.time()))
        print(localtime)
        t = t+1
        if len(wrong) == 0:
            break;
        wrongs = wrong
        wrong = []
        dataset=pd.read_csv('ICML.csv')
        for i in range(len(wrongs)):
            try:               
                link = lists[wrongs[i]].replace('\n','');
                print(link+'  start!')
                if dataset[dataset.link == link].notnull()['abstract'].size != 0:
                    continue
                driver.get(lists[wrongs[i]].replace('\n',''))
                html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
                soup = BeautifulSoup(html, 'html.parser')
                year = str(year)
                title = ''
                name = ''
                pdf = ''
                abstract = ''
                for thing in soup.find_all(class_='post-content'):
                    title = thing.h1.text
                    for names in thing.find_all(id='authors'):
                        name = names.text.replace('\n','').replace(';','').strip()
                    for abstracts in thing.find_all(id='abstract'):
                        abstract = abstracts.text.replace('\n','').strip()
                pattern = re.compile(r'http://proceedings.mlr.press/[0-9a-zA-Z]+/[0-9a-zA-Z/]+.pdf')
                result1 = pattern.findall(html)
                pdf = result1[0]
                dataset = dataset.append([{'year':year,'title':title,'author':name,'abstract':abstract,'link':link,'pdf':pdf}])
                dataset.to_csv('ICML.csv')
                print(lists[wrongs[i]].replace('\n','') + ' success!!!   '+'title is '+ title)
            except:
                wrong.append(wrongs[i])
                print('something went wrong with : ' + lists[wrongs[i]].replace('\n',''))
                continue

In [ ]:
spider_ICML(2019,'http://proceedings.mlr.press/v97/')
spider_ICML(2018,'http://proceedings.mlr.press/v80/')
spider_ICML(2017,'http://proceedings.mlr.press/v70/')
spider_ICML(2016,'http://proceedings.mlr.press/v48/')
spider_ICML(2015,'http://proceedings.mlr.press/v37/')
spider_ICML(2014,'http://proceedings.mlr.press/v32/')
spider_ICML(2013,'http://proceedings.mlr.press/v28/')